In [1]:
import numpy as np
log = np.log
array = np.array
from scipy.optimize import root
from PC_SAFT.PC_SAFT import BPT


In [64]:
MWs_l = np.array([44.01, 61.08, 18.02]) / 1000  # kg/mol

def liquid_density(Tl, x):

    x_CO2, x_MEA, x_H2O = x

    MWT_l = sum([x[i] * MWs_l[i] for i in range(len(x))])

    a1, b1, c1 = [-5.35162e-7, -4.51417e-4, 1.19451]
    a2, b2, c2 = [-3.2484e-6, 0.00165, 0.793]

    V_MEA = MWs_l[1]*1000 / (a1 * Tl ** 2 + b1 * Tl + c1)  # mL/mol
    V_H2O = MWs_l[2]*1000 / (a2 * Tl ** 2 + b2 * Tl + c2)   # mL/mol

    a, b, c, d, e = 10.57920122, -2.020494157, 3.15067933, 192.0126008, -695.3848617


    V_CO2 = a + (b + c * x_MEA) * x_MEA * x_H2O + (d + e * x_MEA) * x_MEA * x_CO2

    V_l = V_CO2 * x_CO2 + x_MEA * V_MEA + x_H2O * V_H2O # Liquid Molar Volume (mL/mol)
    V_l = V_l*1e-6  # Liquid Molar Volume (mL/mol --> m3/mol)

    rho_mol_l = V_l**-1  # Liquid Molar Density (m3/mol --> mol/m3)

    return rho_mol_l

def solve_ChemEQ_log(α, w_MEA, Tl):
    
    MW_CO2 = 44.01/1000
    MW_MEA = 61.08/1000
    MW_H2O = 18.02/1000
    
    def get_x(α, w_MEA):
    
        x_MEA = ((1 + α + (MW_MEA/MW_H2O))*(1-w_MEA)/w_MEA)**-1
        x_CO2 = x_MEA*α
        x_H2O = 1 - x_CO2 - x_MEA
    
        return [x_CO2, x_MEA, x_H2O]
    
    x = get_x(α, w_MEA)
    
    rho_mol_l = liquid_density(Tl, x)
    
    Cl = [x[i] * rho_mol_l for i in range(len(x))]
    
    Cl_MEA_t = Cl[1]
    Cl_H2O_t = Cl[2]
 
    C1 = 132.889, -13445.9, -22.4773, 0
    C2 = 231.465, - 12092.10, - 36.7816, 0
    C3 = 216.049, -12431.70, -35.4819, 0
    C4 = .79960, -8084.81, 0, 0
    C5 = 1.282562, -3456.179, 0, - 0.007484
    

    def f_logK(C, T):
        C1, C2, C3, C4 = C
        return np.exp(C1 + C2 / T + C3 * log(T) + C4 * T)

    K1 = f_logK(C1, Tl)
    K2 = f_logK(C2, Tl)

    K3 = f_logK(C3, Tl)
    K4 = f_logK(C4, Tl)
    K5 = f_logK(C5, Tl)
    
    scale = 1e6

    def f(z):
        Cl_CO2, Cl_MEA, Cl_MEAH, Cl_MEACOO, Cl_HCO3, Cl_H3O, Cl_OH, Cl_CO3 = z

        # Kee1 = log(Cl_H3O) + log(Cl_OH)
        # Kee2 = log(Cl_HCO3)+log(Cl_HCO3) - log(Cl_HCO3)
        # Kee3 = log(Cl_H3O) +log(Cl_CO3) - log(Cl_HCO3)
        # Kee4 = log(Cl_HCO3)+log(Cl_MEA) - log(Cl_MEACOO)
        # Kee5 = log(Cl_H3O)+log(Cl_MEA) - log(Cl_MEAH)
        
        Kee1 = Cl_H3O*Cl_OH
        Kee2 = Cl_HCO3*Cl_HCO3 / Cl_HCO3
        Kee3 = Cl_H3O*Cl_CO3 / Cl_HCO3
        Kee4 = Cl_HCO3*Cl_MEA / Cl_MEACOO
        Kee5 = Cl_H3O*Cl_MEA / Cl_MEAH

        
        eq1 = (Kee1 - K1)*scale
        eq2 = (Kee2 - K2)*scale
        eq3 = (Kee3 - K3)*scale
        eq4 = (Kee4 - K4)*scale
        eq5 = (Kee5 - K5)*scale
        eq6 = Cl_MEA_t - (Cl_MEA + Cl_MEAH - Cl_MEACOO)
        eq7 = Cl_MEA_t*α - (Cl_CO2 - Cl_HCO3 + Cl_CO3 + Cl_MEACOO)
        eq8 = Cl_H3O + Cl_MEAH - Cl_MEACOO - Cl_HCO3 - 2*Cl_CO3 - Cl_OH

        eqs = array([eq1, eq2, eq3, eq4, eq5, eq6, eq7, eq8])
        float_formatter = "{:.4f}".format
        np.set_printoptions(formatter={'float_kind':float_formatter})
        # print(z)
        # print(eqs)
        
        return eqs

    guesses = array([0.1, 4000, 2000, 2000, 50,  10, .01, .01])
    result = root(f, guesses)
    Cl =  array(result.x)
    print(Cl/sum(Cl))
    Cl_CO2, Cl_MEA = Cl[:2]
    x = ([Cl[0], Cl[1], Cl_H2O_t])/np.sum([Cl[0], Cl[1], Cl_H2O_t])
    
    print(x)
    print(result.success)

    return x

x = solve_ChemEQ_log(.3004, .30, 393.15)
x = x[:3]/sum(x[:3])
print(x)


[-0.5264 0.2958 0.6153 0.6153 0.0000 0.0000 -0.0000 0.0000]
[-0.2008 0.1128 1.0879]
True
[-0.2008 0.1128 1.0879]


In [77]:

x = np.array([1.14843E-06, 0.029480076, 0.892089614])
# x = x/sum(x)
# 2B Association Scheme
m = np.array([2.0729, 3.0353, 1.9599])  # Number of segments
σ = np.array([2.7852, 3.0435, 2.362])  # Temperature-Independent segment diameter σ_i (Aᵒ)
ϵ_k = np.array([169.21, 277.174, 279.42]) # Depth of pair potential / Boltzmann constant (K)
k_ij = np.array([[0.0, .16, .065],
                 [.16, 0.0, -.18],
                 [.065, -.18, 0.0]])
κ_AB = np.array([0, .037470, .2039])
ϵ_AB_k = np.array([0, 2586.3, 2059.28])

T = 393.15
yg = [.06, .01, .93]
Pg = 700000
y, P = BPT(T, x, m, σ, ϵ_k, k_ij, yg, Pg, κ_AB=κ_AB, ϵ_AB_k=ϵ_AB_k,)
print(y, P)


[0.0001 0.0002 0.9998] 3843057.035551169


In [78]:
!pip install reportlab
from reportlab.pdfgen import canvas
 
def convert_to_pdf_reportlab(input_file, output_pdf):
    pdf_canvas = canvas.Canvas(output_pdf)
 
    with open(input_file, 'r') as python_file:
        content = python_file.read()
        pdf_canvas.drawString(100, 800, content)
 
    pdf_canvas.save()
 
# Example usage:
convert_to_pdf_reportlab('PC_SAFT.py', 'PC_LAB.pdf')
print("PDF is Saved Successfully")


[notice] A new release of pip is available: 24.0 -> 24.1
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------------------------- 0.0/1.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/1.9 MB 640.0 kB/s eta 0:00:04
   - -------------------------------------- 0.1/1.9 MB 812.7 kB/s eta 0:00:03
   ----- ---------------------------------- 0.3/1.9 MB 2.9 MB/s eta 0:00:01
   --------------- ------------------------ 0.7/1.9 MB 5.9 MB/s eta 0:00:01
   ------------------------- -------------- 1.2/1.9 MB 7.1 MB/s eta 0:00:01
   ------------------------------------ --- 1.8/1.9 MB 8.7 MB/s eta 0:00:01
   ---------------------------------------- 1.9/1.9 MB 8.2 MB/s eta 0:00:00


UnicodeDecodeError: 'charmap' codec can't decode byte 0x81 in position 3443: character maps to <undefined>

In [67]:
print(y[0]*P)

281.91266757481253


In [76]:
R = 8.314 # J/mol-K
T = 300
P = .3*101325 # Pa
B = -1.96/1000
Cv = 67
Cp = 77.3
MW = .058
V = R*T/P + B
print(np.sqrt((V)**2*Cp*R*T/(MW*Cv*(V - B)**2)))

217.42228581608228
